# EE399 HW4
## Ziwen


https://github.com/ZiwenLi0325/EE399.git

In [38]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import torch
from torch import nn, optim
from torch.optim import Adam
from sklearn.metrics import mean_squared_error

In [60]:
dt = 0.01
T = 8
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28

# Define activation functions
def logsig(x):
    return 1 / (1 + torch.exp(-x))

def radbas(x):
    return torch.exp(-torch.pow(x, 2))

def purelin(x):
    return x

# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = purelin(self.fc1(x))
        x = purelin(self.fc2(x))
        x = purelin(self.fc3(x))
        return x

class FeedForwardNN(nn.Module):
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.bn = nn.BatchNorm1d(hidden_size)  # add batch normalization

    def forward(self, x):
        x, _ = self.rnn(x.unsqueeze(1))  # add an extra dimension for timesteps
        x = self.fc(self.bn(x.squeeze(1)))  # apply batch normalization before fc
        return x
    
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x.unsqueeze(1))  # add an extra dimension for timesteps
        x = self.fc(x.squeeze(1))  # remove the timesteps dimension
        return x


class ESN(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, alpha=0.5):
        super(ESN, self).__init__()
        self.input_weights = nn.Parameter(torch.randn(reservoir_size, input_size) / np.sqrt(input_size), requires_grad=False)
        self.reservoir_weights = nn.Parameter(torch.randn(reservoir_size, reservoir_size) / np.sqrt(reservoir_size), requires_grad=False)
        self.output_weights = nn.Linear(reservoir_size, output_size)

        spectral_radius = np.max(np.abs(np.linalg.eigvals(self.reservoir_weights.detach().numpy())))
        self.reservoir_weights.data = self.reservoir_weights.data / spectral_radius * alpha
        self.reservoir_size = reservoir_size

    def forward(self, input):
        reservoir_state = torch.zeros(input.size(0), input.size(1), self.reservoir_size, dtype=torch.float32, device=input.device)
        reservoir_state[:, 0, :] = torch.tanh(torch.mm(input[:,0,:], self.input_weights.t()))  # initialize reservoir state at t=0
        for t in range(1, input.size(1)):  # start loop from t=1
            reservoir_state[:, t, :] = torch.tanh(torch.mm(input[:,t,:], self.input_weights.t()) + torch.mm(reservoir_state[:, t-1, :], self.reservoir_weights.t()))
        output = self.output_weights(reservoir_state)
        return output

    



# train for rho = 10, 28, 40

## nn

In [29]:
# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []
for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

# Initialize the model and optimizer
model_nn_10 = Net()
optimizer = Adam(model_nn_10.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(30):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_nn_10(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    print('For NN : Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))


For NN : Epoch: 0, Loss: 262.59683
For NN : Epoch: 1, Loss: 233.40649
For NN : Epoch: 2, Loss: 205.98650
For NN : Epoch: 3, Loss: 180.33591
For NN : Epoch: 4, Loss: 156.46880
For NN : Epoch: 5, Loss: 134.41217
For NN : Epoch: 6, Loss: 114.19106
For NN : Epoch: 7, Loss: 95.82480
For NN : Epoch: 8, Loss: 79.32680
For NN : Epoch: 9, Loss: 64.70339
For NN : Epoch: 10, Loss: 51.95095
For NN : Epoch: 11, Loss: 41.05153
For NN : Epoch: 12, Loss: 31.96740
For NN : Epoch: 13, Loss: 24.63585
For NN : Epoch: 14, Loss: 18.96352
For NN : Epoch: 15, Loss: 14.82186
For NN : Epoch: 16, Loss: 12.04458
For NN : Epoch: 17, Loss: 10.42864
For NN : Epoch: 18, Loss: 9.73999
For NN : Epoch: 19, Loss: 9.72519
For NN : Epoch: 20, Loss: 10.12858
For NN : Epoch: 21, Loss: 10.71262
For NN : Epoch: 22, Loss: 11.27713
For NN : Epoch: 23, Loss: 11.67324
For NN : Epoch: 24, Loss: 11.80956
For NN : Epoch: 25, Loss: 11.65015
For NN : Epoch: 26, Loss: 11.20623
For NN : Epoch: 27, Loss: 10.52380
For NN : Epoch: 28, Loss:

## FeedForwardNN: rho = 10

In [30]:

# Initialize the model and optimizer
model_FFNN_10 = FeedForwardNN()
optimizer = Adam(model_FFNN_10.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(30):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_FFNN_10(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    print('For FeedForwardNN: Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

For FeedForwardNN: Epoch: 0, Loss: 362.94348
For FeedForwardNN: Epoch: 1, Loss: 353.25021
For FeedForwardNN: Epoch: 2, Loss: 343.87408
For FeedForwardNN: Epoch: 3, Loss: 334.75436
For FeedForwardNN: Epoch: 4, Loss: 325.90726
For FeedForwardNN: Epoch: 5, Loss: 317.31183
For FeedForwardNN: Epoch: 6, Loss: 308.90826
For FeedForwardNN: Epoch: 7, Loss: 300.69992
For FeedForwardNN: Epoch: 8, Loss: 292.65540
For FeedForwardNN: Epoch: 9, Loss: 284.75180
For FeedForwardNN: Epoch: 10, Loss: 276.97278
For FeedForwardNN: Epoch: 11, Loss: 269.32770
For FeedForwardNN: Epoch: 12, Loss: 261.78406
For FeedForwardNN: Epoch: 13, Loss: 254.30779
For FeedForwardNN: Epoch: 14, Loss: 246.85889
For FeedForwardNN: Epoch: 15, Loss: 239.46001
For FeedForwardNN: Epoch: 16, Loss: 232.12520
For FeedForwardNN: Epoch: 17, Loss: 224.86005
For FeedForwardNN: Epoch: 18, Loss: 217.67726
For FeedForwardNN: Epoch: 19, Loss: 210.59267
For FeedForwardNN: Epoch: 20, Loss: 203.60683
For FeedForwardNN: Epoch: 21, Loss: 196.6944

## SimpleRNN: rho = 10

In [72]:
# Generate training data for rho=10, 28, 40
training_input = []
training_output = []

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
model_rnn = SimpleRNN(3, 50, 3)  # input size = 3, hidden size = 50, output size = 3
optimizer = Adam(model_rnn.parameters(), lr=0.01)  # adjust the learning rate if necessary

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(30):  # 30 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_rnn(training_input_torch)
    loss = criterion(output, training_output_torch)

    # Add L2 regularization
    l2_reg = None
    for W in model_rnn.parameters():
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    loss += 0.01 * l2_reg

    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model_rnn.parameters(), max_norm=1)

    optimizer.step()    # Does the update

    # Update learning rate
    scheduler.step()

    print('For SimpleRNN: Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

c:\Users\ZiwenLi\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


For SimpleRNN: Epoch: 0, Loss: 552.62665
For SimpleRNN: Epoch: 1, Loss: 543.49152
For SimpleRNN: Epoch: 2, Loss: 535.11523
For SimpleRNN: Epoch: 3, Loss: 527.40399
For SimpleRNN: Epoch: 4, Loss: 520.43207
For SimpleRNN: Epoch: 5, Loss: 513.68231
For SimpleRNN: Epoch: 6, Loss: 506.98502
For SimpleRNN: Epoch: 7, Loss: 500.25912
For SimpleRNN: Epoch: 8, Loss: 493.30066
For SimpleRNN: Epoch: 9, Loss: 485.98740
For SimpleRNN: Epoch: 10, Loss: 479.25558
For SimpleRNN: Epoch: 11, Loss: 472.62903
For SimpleRNN: Epoch: 12, Loss: 465.73642
For SimpleRNN: Epoch: 13, Loss: 458.15955
For SimpleRNN: Epoch: 14, Loss: 450.66479
For SimpleRNN: Epoch: 15, Loss: 443.40033
For SimpleRNN: Epoch: 16, Loss: 436.28262
For SimpleRNN: Epoch: 17, Loss: 428.77496
For SimpleRNN: Epoch: 18, Loss: 420.92102
For SimpleRNN: Epoch: 19, Loss: 412.85147
For SimpleRNN: Epoch: 20, Loss: 404.82715
For SimpleRNN: Epoch: 21, Loss: 396.81866
For SimpleRNN: Epoch: 22, Loss: 388.52243
For SimpleRNN: Epoch: 23, Loss: 380.09171
Fo

## LSTM: rho = 10 

In [71]:
# Initialize the model and optimizer
model_LSTM = LSTM(3, 50, 3)  # input size = 3, hidden size = 50, output size = 3
optimizer = Adam(model_LSTM.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(30):  # 30 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model_LSTM(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    print('For LSTM : Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

For LSTM : Epoch: 0, Loss: 554.13330
For LSTM : Epoch: 1, Loss: 553.48798
For LSTM : Epoch: 2, Loss: 552.84259
For LSTM : Epoch: 3, Loss: 552.19666
For LSTM : Epoch: 4, Loss: 551.54950
For LSTM : Epoch: 5, Loss: 550.90076
For LSTM : Epoch: 6, Loss: 550.24994
For LSTM : Epoch: 7, Loss: 549.59662
For LSTM : Epoch: 8, Loss: 548.94055
For LSTM : Epoch: 9, Loss: 548.28125
For LSTM : Epoch: 10, Loss: 547.61853
For LSTM : Epoch: 11, Loss: 546.95209
For LSTM : Epoch: 12, Loss: 546.28162
For LSTM : Epoch: 13, Loss: 545.60669
For LSTM : Epoch: 14, Loss: 544.92688
For LSTM : Epoch: 15, Loss: 544.24176
For LSTM : Epoch: 16, Loss: 543.55084
For LSTM : Epoch: 17, Loss: 542.85400
For LSTM : Epoch: 18, Loss: 542.15106
For LSTM : Epoch: 19, Loss: 541.44232
For LSTM : Epoch: 20, Loss: 540.72839
For LSTM : Epoch: 21, Loss: 540.01007
For LSTM : Epoch: 22, Loss: 539.28857
For LSTM : Epoch: 23, Loss: 538.56519
For LSTM : Epoch: 24, Loss: 537.84082
For LSTM : Epoch: 25, Loss: 537.11621
For LSTM : Epoch: 26, 

## ESN:

In [68]:
# Define the models
model_esn_10 = ESN(3, 50, 3)

# Initialize the optimizer for ESN
optimizer_esn = torch.optim.Adam(model_esn_10.output_weights.parameters(), lr=0.01)

# Add an extra dimension for time step to the input tensor
training_input_torch_time = training_input_torch.unsqueeze(1)

# Train the ESN network
for epoch in range(30):  # match the number of epochs with NN
    optimizer_esn.zero_grad()   # zero the gradient buffers
    output_esn = model_esn_10(training_input_torch_time)
    # Remove the time step dimension from the output for loss calculation
    output_esn = output_esn.squeeze(1)
    loss_esn = criterion(output_esn, training_output_torch)
    loss_esn.backward()
    optimizer_esn.step()    # Does the update
    print('For ESN : Epoch: {}, Loss: {:.5f}'.format(epoch, loss_esn.item()))



For ESN : Epoch: 0, Loss: 553.31342
For ESN : Epoch: 1, Loss: 542.24335
For ESN : Epoch: 2, Loss: 531.35760
For ESN : Epoch: 3, Loss: 520.64752
For ESN : Epoch: 4, Loss: 510.10434
For ESN : Epoch: 5, Loss: 499.71899
For ESN : Epoch: 6, Loss: 489.48404
For ESN : Epoch: 7, Loss: 479.39423
For ESN : Epoch: 8, Loss: 469.44647
For ESN : Epoch: 9, Loss: 459.63950
For ESN : Epoch: 10, Loss: 449.97397
For ESN : Epoch: 11, Loss: 440.45200
For ESN : Epoch: 12, Loss: 431.07632
For ESN : Epoch: 13, Loss: 421.84982
For ESN : Epoch: 14, Loss: 412.77466
For ESN : Epoch: 15, Loss: 403.85260
For ESN : Epoch: 16, Loss: 395.08456
For ESN : Epoch: 17, Loss: 386.47079
For ESN : Epoch: 18, Loss: 378.01105
For ESN : Epoch: 19, Loss: 369.70471
For ESN : Epoch: 20, Loss: 361.55081
For ESN : Epoch: 21, Loss: 353.54852
For ESN : Epoch: 22, Loss: 345.69717
For ESN : Epoch: 23, Loss: 337.99615
For ESN : Epoch: 24, Loss: 330.44519
For ESN : Epoch: 25, Loss: 323.04407
For ESN : Epoch: 26, Loss: 315.79248
For ESN : E

In [80]:
rhos = [17, 35]

def generate_lorenz_data(rho, initial_state=[1, 1, 1], dt=0.01, N=10000, sigma=10, beta=8/3):
    t = np.arange(0, N*dt, dt)
    traj = integrate.odeint(lorenz_deriv, initial_state, t, args=(sigma, beta, rho))
    return traj[:-1, :], traj[1:, :]

for rho in rhos:
    print(f"\nForecasting dynamics for rho = {rho}:")

    # Generate Lorenz system data for rho
    nn_input, nn_output = generate_lorenz_data(rho)

    # Train-test split
    train_frac = 0.8
    split_idx = int(train_frac * len(nn_input))
    train_input, train_output = nn_input[:split_idx], nn_output[:split_idx]
    test_input, test_output = nn_input[split_idx:], nn_output[split_idx:]

    # Convert to torch tensors
    train_input_torch = torch.from_numpy(train_input.astype(np.float32))
    train_output_torch = torch.from_numpy(train_output.astype(np.float32))
    test_input_torch = torch.from_numpy(test_input.astype(np.float32))
    test_output_torch = torch.from_numpy(test_output.astype(np.float32))

    # Define the models
    model1 = model_nn_10
    model2 = model_FFNN_10
    model3 = model_LSTM
    model4 = model_rnn
    model5 = model_esn_10
  

    # Define the loss function
    criterion = nn.MSELoss()

    # Train each model and make predictions
    for model in [model1, model2, model3, model4, model5]:
        # Define the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        # Train the model
        for epoch in range(100):
            optimizer.zero_grad()
            output = model(train_input_torch)
            loss = criterion(output, train_output_torch)
            loss.backward()
            optimizer.step()

        # Make predictions on the test data
        model.eval()
        with torch.no_grad():
            predictions = model(test_input_torch)

        # Compute the mean squared error of the predictions
        mse = mean_squared_error(test_output_torch.detach().numpy(), predictions.detach().numpy())
        print(f"Mean Squared Error for model {model.__class__.__name__}: {mse}")
    


Forecasting dynamics for rho = 17:
Mean Squared Error for model Net: 7.215717050712556e-05
Mean Squared Error for model FeedForwardNN: 4.492899097385816e-05
Mean Squared Error for model LSTM: 9.988838428398594e-05
Mean Squared Error for model SimpleRNN: 0.0012948585208505392


IndexError: too many indices for tensor of dimension 2

In [84]:
for rho in rhos:
    print(f"\nForecasting dynamics for rho = {rho}:")

    # Generate Lorenz system data for rho
    nn_input, nn_output = generate_lorenz_data(rho)

    # Train-test split
    train_frac = 0.8
    split_idx = int(train_frac * len(nn_input))
    train_input, train_output = nn_input[:split_idx], nn_output[:split_idx]
    test_input, test_output = nn_input[split_idx:], nn_output[split_idx:]

    # Convert to torch tensors and add time dimension
    train_input_torch = torch.from_numpy(train_input.astype(np.float32)).unsqueeze(1)
    train_output_torch = torch.from_numpy(train_output.astype(np.float32))
    test_input_torch = torch.from_numpy(test_input.astype(np.float32)).unsqueeze(1)
    test_output_torch = torch.from_numpy(test_output.astype(np.float32))

      # Define the models

    model5 = model_esn_10
  

    # Define the loss function
    criterion = nn.MSELoss()

    # Train each model and make predictions
    for model in [model5]:
        # Define the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        # Train the model
        for epoch in range(100):
            optimizer.zero_grad()
            output = output.squeeze(1)  # remove the time dimension for ESN output
            loss = criterion(output, train_output_torch)
            loss.backward()
            optimizer.step()

        # Make predictions on the test data
        model.eval()
        with torch.no_grad():
            predictions = predictions.squeeze(1)  # remove the time dimension for ESN predictions

        # Compute the mean squared error of the predictions
        mse = mean_squared_error(test_output_torch.detach().numpy(), predictions.detach().numpy())
        print(f"Mean Squared Error for model {model.__class__.__name__}: {mse}")



Forecasting dynamics for rho = 17:


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.